# В этом ноутбуке будем тренировать логистическую регререссию на наших данных

In [38]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

будем использовать нормализованный датасет

In [39]:
#df_train = pd.read_csv("/Volumes/HP P800/itmo/Lidar data/data_train.csv")
df_train =  pd.read_csv("/Volumes/HP P800/itmo/Lidar data/data_simple_preprocess_normalize_all_columns_train.csv")
# df_train =  pd.read_csv("/Volumes/HP P800/itmo/Lidar data/df_kdtree_train.csv")

Сделаем проверку на дисбаланс классов

In [40]:
df_train.groupby(["label"]).count()

,Unnamed: 0,scene_id,x,y,z,intensity,ring,x_normailized,y_normailized,z_normailized,ring_normailized,intensity_normailized
label,,,,,,,,,,,,
0.0,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070
1.0,481210,481210,481210,481210,481210,481210,481210,481210,481210,481210,481210,481210


Как видим дисбаланас классов очень большой, избавимся от него путем раскопирования данных

In [41]:
k = df_train[df_train["label"] == 1.0]

m = len(df_train[df_train["label"] == 0.0]) // len(df_train[df_train["label"] == 1.0])
ost = len(df_train[df_train["label"] == 0.0]) % len(df_train[df_train["label"] == 1.0])

for i in range(0,m-1):
    df_train = df_train.append(k)

df_train = df_train.append(k.head(ost))

/var/folders/bx/60hcl2tn12bb9txf54xp9l9m0000gn/T/ipykernel_764/155822642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(k)
/var/folders/bx/60hcl2tn12bb9txf54xp9l9m0000gn/T/ipykernel_764/155822642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(k)
/var/folders/bx/60hcl2tn12bb9txf54xp9l9m0000gn/T/ipykernel_764/155822642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(k)
/var/folders/bx/60hcl2tn12bb9txf54xp9l9m0000gn/T/ipykernel_764/155822642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(k)
/var/folders/bx/60hcl2tn

In [42]:
df_train.groupby(["label"]).count()

,Unnamed: 0,scene_id,x,y,z,intensity,ring,x_normailized,y_normailized,z_normailized,ring_normailized,intensity_normailized
label,,,,,,,,,,,,
0.0,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070
1.0,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070,4005070


Теперь количество сэмлов в классах равны

Разделиим данные на X и y. Не будем использовать фичи z и ring вместе, потому что они имеют большую корреляцию

In [44]:
X_train = df_train[["x_normailized", "y_normailized", "z_normailized", "intensity_normailized"]]
y_train = df_train[["label"]]

In [45]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

/Users/alexander.savelyev/opt/anaconda3/envs/Snow_recognition/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Выведем roc_auc_score для тренировочных данных

In [52]:
roc_auc_score(y_train, clf.predict_proba(X_train)[:, 1])

0.9110154128547827

Теперь оценим результат roc_auc_score на тестовых данных

In [47]:
#df_test = pd.read_csv("/Volumes/HP P800/itmo/Lidar data/data_test.csv")
df_test = pd.read_csv("/Volumes/HP P800/itmo/Lidar data/data_simple_preprocess_normalize_all_columns_test.csv")
# df_test = pd.read_csv(("/Volumes/HP P800/itmo/Lidar data/df_kdtree_test.csv")

In [48]:
X_test = df_test[["x_normailized", "y_normailized", "z_normailized", "intensity_normailized"]] 
y_test = df_test[["label"]]

In [49]:
roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

0.8931836315132812

Посмотрим на веса классификатора

In [50]:
print(clf.coef_)

[[-5.42831598e-02  6.49432114e-01  1.91181218e+01  2.62015774e-03]]


Как видим z_normailized имеет наибольший вес для данной модели, тогда если значение z большое, то классификатор отнесете его к снежинке с большой вероятностью